# Here goes the title
---

Some description.

*Code made by [You](https://github.com/@you) - YYYY.*

### Import required libraries

In [1]:
# IMPORT CONFIGs
import os
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../scripts')
from config import RAW_PATH, TREAT_PATH, OUTPUT_PATH


# DATA ANALYSIS & VIZ TOOLS
from copy import deepcopy

import pandas as pd
pd.options.display.max_columns = 999

#import plotly
#import plotly.graph_objs as go
#import plotly.offline as offline
#offline.init_notebook_mode(connected=True)

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)


# For changes in .py
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')




import numpy as np


import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

import requests
import json
import xmltodict
import re
import urllib.request as urllib2
import io

Populating the interactive namespace from numpy and matplotlib


### Import data

- http://www.seade.gov.br/lista-produtos/

- Get the data from http://doc.seade.gov.br/index.php/API-SEADE

### Fonte

In [3]:
url = "http://api-imp.seade.gov.br/v1/fonte"
r = requests.get(url).json()
df_font = pd.DataFrame.from_records(r['fontes'])

In [4]:
df_font.head()

,codigo,nome
0,1,Banco Central do Brasil – Bacen.
1,999,Fundação Seade.
2,3,Instituto Brasileiro de Geografia e Estatística – IBGE.
3,4,Instituto Geográfico e Cartográfico – IGC.
4,5,Telecomunicações Brasileiras S.A. – Telebras.


### Variavel

Use o formato abaixo para buscar uma variável por requisição.

- Path: /variavel
- Parâmetros: código

Para buscar todas as variáveis, é necessário informar os parâmetros de paginação, conforme a sintaxe abaixo.
- Path: /variavel?offset={inicial}&limit={limite}

In [5]:
url =  "http://api-imp.seade.gov.br/v1/variavel?offset=0&limit=100000"
r = requests.get(url).json()
df_var = pd.DataFrame.from_records(r['variavel'])

In [6]:
var_list = list(df_var.keys())

### Localidade

In [7]:
url = "http://api-imp.seade.gov.br/v1/localidade"
r = requests.get(url).json()
df_loc = pd.DataFrame.from_records(r['localidades'])

In [8]:
df_loc.head()

,codigo,codigo_pai,nivel,nome,nome_pai
0,903,1000,Aglomeração Urbana,Aglomeração Urbana de Franca,Total do Estado de São Paulo
1,901,1000,Aglomeração Urbana,Aglomeração Urbana de Jundiaí,Total do Estado de São Paulo
2,902,1000,Aglomeração Urbana,Aglomeração Urbana de Piracicaba,Total do Estado de São Paulo
3,801,503,Distritos,Água Rasa,São Paulo
4,802,503,Distritos,Alto de Pinheiros,São Paulo


### Notas

In [9]:
url = "http://api-imp.seade.gov.br/v1/nota"
r = requests.get(url).json()
df_notas = pd.DataFrame.from_records(r['notas'])

In [10]:
df_notas.head()

,codigo,descricao
0,1,"Os terminais incluem os terminais de uso público e não incluem os telefones celulares. Em 1980, as totalizações para o Estado, Região Administrativa de Bauru e Região de Governo de Jaú incluem os dados referentes aos municípios de Barra Bonita e Igaraçu do Tietê. Em 1985, as totalizações para o Estado e Região Metropolitana incluem os dados referentes aos municípios de Vargem Grande Paulista e Cotia. Apesar da inexistência de terminais telefônicos, em 1993, nos municípios novos de Torre de Pedra, Barra do Chapéu, Bom Sucesso de Itararé, Canitar, Nova Campina, Iaras, Itaoca e Itapirapuã Paulista, cabe ressaltar que tal fato não significa que estas localidades não tenham serviço telefônico. Na verdade, estes municípios possuem à sua disposição, pelo menos 1 telefone público ligado a um terminal contabilizado no município do qual se originou o desmembramento. Em 1993, o município de Águas de Santa Bárbara inclui os terminais telefônicos do município de Iaras; Porangaba inclui os terminais de Torre de Pedra; Apiai inclui os terminais de Barra do Chapéu e Itaoca; Itapeva inclui os terminais de Nova Campina; Itararé inclui os terminais de Bom Sucesso de Itararé; Ribeira inclui os terminais de Itapirapuã Paulista; Chavantes inclui os terminais de Canitar. Em 1994, o município de Águas de Santa Bárbara inclui os terminais telefônicos do município de Iaras; Porangaba inclui os terminais de Torre de Pedra; Itapeva inclui os terminais de Nova Campina; Itararé inclui os terminais de Bom Sucesso de Itararé; Artur Nogueira inclui os terminais de Engenheiro Coelho. Em 1995, Itararé inclui os terminais de Bom Sucesso de Itararé; Paranapuã inclui os terminais de Mesópolis; Assis inclui os terminais de Tarumã; Santa Cruz do Rio Pardo inclui os terminais de Espírito Santo do Turvo; Cajobi inclui os terminais de Embaúba. Em 1996, Santa Cruz do Rio Pardo inclui os terminais telefônicos do município de Espírito Santo do Turvo. Em 1999 o cálculo do coeficiente para as Regiões de Governo, Regiões Administrativas e Estado foi efetuado sem os dados de população e de terminais telefônicos referentes aos municípios que são operados pela Companhia de Telefones do Brasil Central - CTBC ou Centrais Telefônicas de Ribeirão Preto - CETERP: Tapiratiba, pertencente a RG de São João da Boa Vista; Altinópolis, Brodósqui, Cajuru, Cássia dos Coqueiros, Jardinópolis, Ribeirão Preto, Santo Antônio da Alegria, Guatapará e Santa Cruz 2da Esperança, pertencentes a RG de Ribeirão Preto; Paulistânia, pertencente a RG de Bauru; Fernão, pertencente a RG de Marília; Colômbia e Guaíra, pertencentes a RG de Barretos; Aramina, Batatais, Buritizal, Franca, Guará, Ituverava, Miguelópolis e Ribeirão Corrente, pertencentes a RG de Franca; Ipuã, Morro Agudo, Nuporanga, Orlândia, Sales de Oliveira e São Joaquim da Barra, municípios que compõem a RG de São Joaquim da Barra."
1,2,"Os dados referem-se aos municípios de acordo com a divisão administrativa do Estado de São Paulo vigente no respectivo período:<BR>- até 1982, 571 municípios;<BR>- de 1983 a 1992, 572 municípios;<BR>- de 1993 a 1996, 625 municípios;<BR>- a partir 1997, 645 municípios.<BR>"
2,3,"A maior variabilidade nas taxas, em alguns municípios, pode decorrer do número reduzido de nascidos vivos e nascidos mortos, ocorridos em cada ano considerado."
3,4,"Refere-se a vínculo empregatício remunerado, efetivamente ocupado em 31/12,referente a trabalhadores com carteira de trabalho assinada (regime da Consolidação das Leis do Trabalho - CLT), estatutários (funcionários públicos) e trabalhadores avulsos, temporários e outros, desde que formalmente contratados.Estes dados são informados pelos estabelecimentos quando da elaboração da Relação Anual de Informações Sociais - RAIS, do Ministério do Trabalho. Deve-se observar que: mesmo indivíduo pode estar acumulando, na data de referência, mais de um emprego (ANUÁRIO RAIS 1992). Esta diferença deve-se ao fato de que o levantamento é feito a partir dos estabele

### Perfil Municipal - informações do município

In [11]:
url = "http://api-imp.seade.gov.br/v1/infmun/503"
r = requests.get(url).json()
# df_notas = pd.DataFrame.from_records(r['notas'])

### Perfil Municipal - dados

In [13]:
url = "http://api-imp.seade.gov.br/v1/perfil/503"
r = requests.get(url).json()
df_dados = pd.DataFrame.from_records(r['perfil'])

In [14]:
r['perfil'][0]

{'codigo_localidade': '503',
 'localidade': 'São Paulo',
 'codigo_localidade_pai': '681',
 'nome_localidade_pai': 'Região Metropolitana de São Paulo',
 'codigo_variavel': '1',
 'variavel': 'Crédito Rural [Série interrompida]',
 'unidade': '(Em reais de 2018)',
 'periodo': '1985/1989-2011',
 'ano': {'2011': '2.754.299.963'}}

In [15]:
df_dados.head()

,ano,codigo_localidade,codigo_localidade_pai,codigo_variavel,localidade,nome_localidade_pai,periodo,unidade,variavel
0,{'2011': '2.754.299.963'},503,681,1,São Paulo,Região Metropolitana de São Paulo,1985/1989-2011,(Em reais de 2018),Crédito Rural [Série interrompida]
1,{'2011': '1.890.486.925'},503,681,2,São Paulo,Região Metropolitana de São Paulo,1985/1989-2011,(Em reais de 2018),Crédito Rural Agricultura [Série interrompida]
2,{'2011': '863.813.040'},503,681,3,São Paulo,Região Metropolitana de São Paulo,1985/1989-2011,(Em reais de 2018),Crédito Rural Pecuária [Série interrompida]
3,{'2001': 'Nossa Senhora da Penha de Franca'},503,681,4,São Paulo,Região Metropolitana de São Paulo,2001,Não possui unidade,Santo Padroeiro
4,"{'2019': '1.521,11'}",503,681,5,São Paulo,Região Metropolitana de São Paulo,1980-2019,(Em km2),Área


### Dados

- Path: /dados
- Parâmetros: localidade/variável/ano

* É obrigatório a informação de no mínimo dois parâmetros.

* Coloque zero quando desejar trazer todos os dados de uma categoria.

In [17]:
url = "http://api-imp.seade.gov.br/v1/dados/554/0/2011"
r = requests.get(url).json()
df = pd.DataFrame.from_records(r['dados'])





df.head()

,ano,codigo_localidade,codigo_variavel,localidade,nome_localidade_pai,periodo,unidade,variavel
0,{'2011': '130.963'},554,1,Ubatuba,Região de Governo de Caraguatatuba,1985/1989-2011,(Em reais de 2018),Crédito Rural [Série interrompida]
1,{'2011': '-'},554,2,Ubatuba,Região de Governo de Caraguatatuba,1985/1989-2011,(Em reais de 2018),Crédito Rural Agricultura [Série interrompida]
2,{'2011': '130.963'},554,3,Ubatuba,Região de Governo de Caraguatatuba,1985/1989-2011,(Em reais de 2018),Crédito Rural Pecuária [Série interrompida]
3,"{'2011': '723,83'}",554,5,Ubatuba,Região de Governo de Caraguatatuba,1980-2019,(Em km2),Área
4,{'2011': '79.706'},554,55,Ubatuba,Região de Governo de Caraguatatuba,1980-2019,Não possui unidade,População


In [18]:
df['dado'] = df['ano'].apply(lambda x: list(x.values())[0])
df['ano']  = df['ano'].apply(lambda x: list(x.keys())[0])

### Get Data

In [22]:
# var_list
locs = list(df_loc['codigo'].unique())
anos = [i for i in range(1980,2021)]

In [34]:
print('localidades = {}'.format(len(locs)))
print('variaveis   = {}'.format(len(var_list)))
print('anos        = {}'.format(len(anos)))
print('Total        = {}'.format(len(locs)*len(var_list)*len(anos)))


localidades = 808
variaveis   = 1347
anos        = 41
Total        = 44623416


In [35]:
def get_all_cities(locs,anos):
    ### Generate a Clear Dataframe
    df = pd.DataFrame([1])
    df['ano']                 = 'missing'
    df['codigo_localidade']   = 'missing'
    df['codigo_variavel']     = 'missing'
    df['localidade']          = 'missing'
    df['nome_localidade_pai'] = 'missing'
    df['periodo']             = 'missing'
    df['unidade']             = 'missing'
    df['variavel']            = 'missing'

    df['loc'] = 'teste'
    df['year']= 'teste'
    df = df.drop(columns=[0])
    df.to_csv("dados_consolidados_2.csv", index=False, encoding='utf-8')
    
    count = 0
    for localidade in locs:
#     print(count,localidade)
        count+=1
        for ano in anos:
            url = "http://api-imp.seade.gov.br/v1/dados/{}/0/{}".format(localidade,ano)
    #           r = requests.get(url).json()
    #             url = "http://api-imp.seade.gov.br/v1/dados/0/2/2010"
            r = requests.get(url).json()
            try:
                df = pd.DataFrame.from_records(r['dados'])
                df['loc'] = localidade
                df['year']= ano

            except:
                df = pd.DataFrame([1])
                df['ano']                 = 'missing'
                df['codigo_localidade']   = 'missing'
                df['codigo_variavel']     = 'missing'
                df['localidade']          = 'missing'
                df['nome_localidade_pai'] = 'missing'
                df['periodo']             = 'missing'
                df['unidade']             = 'missing'
                df['variavel']            = 'missing'
                df['loc'] = localidade
                df['year']= ano
                df = df.drop(columns=[0])

    #         df_final = pd.concat([df_final,df], axis=0)
    #         df.to_csv("dados_consolidados_3.csv", index=False, encoding='utf-8')

            df.to_csv("dados_consolidados_2.csv", index=False, encoding='utf-8', mode='a', header=False)

In [36]:
df_total_1 = pd.read_csv('dados_consolidados.csv')

In [37]:
df_total_2 = pd.read_csv('dados_consolidados_2.csv')

In [38]:
df_final = pd.concat([df_total_1,df_total_2],axis=0)

In [1]:
# df_final.to_csv('dados_consolidados_final.csv', index=False, encoding='utf-8', chunksize=20000)

# ICMS


- Colunas capturadas 'Meses', 'ICMS (\*\*)', 'IPVA (\*\*\*)', 'Fund.Exp-IPI (\*\*)', 'Comp. (\*)', 'Total'

- (Valores expressos em Reais)

- (*)	Compensação Financeira sobre Exploração de Gás, Energia Elétrica, Óleo Bruto, Xisto Betuminoso de acordo com a Lei 7.990 de 28/12/89

- (**)	Até fevereiro/2007, valores com desconto de 15% referente à transferência para o FUNDEF, de acordo com a Lei 9.424 de 24/12/1996. A partir de março de 2007 valores líquidos, descontados o montante transferido para o FUNDEB, de acordo com a Lei 11.494 de 20/06/2007. No período de março a dezembro de 2007, valores com desconto de 16,66%; no ano de 2008, valores com desconto de 18,33%; e a partir de janeiro de 2009, valores com desconto de 20%.

- (***)	Valor referente à receita bruta sem desconto do FUNDEB, que, de acordo com a Lei 11.494 de 20/06/2007, corresponde a 6,66% no ano de 2007, 13,33% no ano de 2008 e 20,00% a partir do ano de 2009. Inclui, quando for o caso, receita de PPD – Programa de Parcelamento de Débitos.

- Obs:	As divergências de centavos entre a soma das parcelas e o total decorrem de erro de aproximação




- [Saiba mais sobre o Repasse de Tributos](https://www.fazenda.sp.gov.br/RepasseConsulta/Consulta/r1.asp)

- Observação: As atualizações das contas bancárias para o repasse da cota-parte da arrecadação de IPVA e das receitas decorrentes do recebimento das multas de trânsito de convênio auto-gestão deverão ser informadas diretamente à Federação Brasileira dos Bancos - FEBRABAN, por meio de ofício assinado pelo Prefeito ou Secretário de Finanças ou Fazenda, com os dados discriminados no [Comunicado DA 86/2010](https://www.fazenda.sp.gov.br/RepasseConsulta/Consulta/comunicado.shtm)

## Bolsa Familia



In [39]:
years = [i+1 for i in range(2003,2019)]

for i in range(len(years)):
    url = "http://aplicacoes.mds.gov.br/sagi/servicos/misocial?q=*&fq=anomes_s:{}*&fq=tipo_s:mes_mu&wt=csv&fl=ibge:codigo_ibge,anomes:anomes_s,qtd_familias_beneficiarias_bolsa_familia,valor_repassado_bolsa_familia&rows=10000000&sort=anomes_s%20asc,%20codigo_ibge%20asc".format(years[i])
    df = pd.read_csv(url)
    
    if i==0:
        df.head(0).to_csv('bolsa_familia.csv', index=False, encoding='utf-8')
        
    df.to_csv('bolsa_familia.csv', mode = 'a', header=False, index=False, encoding='utf-8')

In [52]:
years = [i+1 for i in range(2013,2019)]

for i in range(len(years)):
    url = "http://aplicacoes.mds.gov.br/sagi/servicos/misocial?fq=anomes_s:{}*&fq=tipo_s:mes_mu&indent=on&q=*:*&wt=csv&rows=10000000&sort=codigo_ibge%20asc,%20anomes_s%20asc&fl=ibge:codigo_ibge,siglauf:sigla_uf,anomes:anomes_s,qtd_ben_bas:tot_ben_bas_i,qtd_ben_var:tot_ben_bas_var_i,qtd_ben_bvj:tot_ben_bvj_i,qtd_ben_bvn:tot_ben_bvn_i,qtd_ben_bvg:tot_ben_bvg_i,qtd_ben_bsp:tot_ben_bsp_i".format(years[i])
    df = pd.read_csv(url)
    
    if i==0:
        df.head(0).to_csv('bolsa_familia_more_variables.csv', index=False, encoding='utf-8')
        
    df.to_csv('bolsa_familia_more_variables.csv', mode = 'a', header=False, index=False, encoding='utf-8')

## BPC

In [45]:
years = [i+1 for i in range(2003,2019)]

for i in range(len(years)):
    url = "http://aplicacoes.mds.gov.br/sagi/servicos/misocial?q=*&fq=anomes_s:{}*&fq=tipo_s:mes_mu&wt=csv&omitHeader=true&fl=ibge:codigo_ibge,anomes:anomes_s,bpc_ben:bpc_ben_i,bpc_pcd_ben:bpc_pcd_ben_i,bpc_idoso_ben:bpc_idoso_ben_i,bpc_pcd_val:bpc_pcd_val_s,bpc_idoso_val:bpc_idoso_val_s,bpc_val:bpc_val_s&rows=100000000&sort=anomes_s%20asc,%20codigo_ibge%20asc".format(years[i])
    df = pd.read_csv(url)
    
    if i==0:
        df.head(0).to_csv('bpc.csv', index=False, encoding='utf-8') 
    
    df.to_csv('bpc.csv', mode = 'a', header=False, index=False, encoding='utf-8')